# Проект: вариант 4



# Задание 1

Представьте, что вы аналитик в компании, которая разрабатывает приложение для обработки и оформления фотографий в формате Stories (например, для дальнейшего экспорта в Instagram Stories). Был проведен A/B тест: тестовой группе предлагалась новая модель оплаты коллекций шаблонов, контрольной – старая механика. Ваша основная задача: проанализировать итоги эксперимента и решить, нужно ли выкатывать новую модель на остальных пользователей.

В ходе отчета обоснуйте выбор метрик, на которые вы обращаете внимание. Если различия есть, то объясните, с чем они могут быть связаны и являются ли значимыми.

Данные:

active_users – информация о пользователях, которые посещали приложение во время эксперимента. 

groups – разбивка пользователей на контрольную (А) и тестовую (В) группы. 

purchases – данные о транзакциях (оплатах) пользователей приложения во время эксперимента 

 

#### Формулируем гипотезу в А/В тесте: 
Проводя анализ, мы смотрим, как влияет алгоритм на доход.

Ключевые метрики - средний/медианный показатель дохода по платящим пользователям ARPPU (Средний доход на платящего пользователя) и  CR конверсия (отношение числа пользователей, которые выполнили какое-либо целевое действие к общему числу пользователей)

Н0 - нулевая гипотеза о том, что изменений в использовании новой модели оплаты коллекций шаблонов,влияющих на ARPPU и конверсию, не имеется (то есть эффекта нет)

Н1 - гипотеза о том, что изменения в новой модели оплаты, влияющие на ARPPU и CR, имеются. 


In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats 
import scipy.stats as ss
from scipy.stats import norm, mannwhitneyu
plt.style.use('ggplot')

In [2]:
df_act_users = pd.read_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-l-petrova/Final Project/Проект_4_active_users.csv', sep=',')

In [3]:
df_act_users .head()

,user_id,country,platform,sex
0,8029,Russia,ios,male
1,1735,Sweden,ios,male
2,7697,Russia,android,male
3,1421,Sweden,android,female
4,1704,Russia,ios,male


In [4]:
df_act_users.shape[0]

8341

In [5]:
df_act_users.dtypes

user_id      int64
country     object
platform    object
sex         object
dtype: object

In [6]:
# проверим данные на присутствие пропцщенных значений
df_act_users.isna().sum()

user_id     0
country     0
platform    0
sex         0
dtype: int64

In [7]:
df_groups = pd.read_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-l-petrova/Final Project/Проект_4_groups.csv', sep=',')

In [8]:
df_groups.head()

,user_id,group
0,9149,B
1,9150,B
2,9151,A
3,9152,B
4,9153,B


In [9]:
# A - control, B - test groups

In [10]:
df_groups.shape[0]

74576

In [11]:
df_groups.dtypes

user_id     int64
group      object
dtype: object

In [12]:
df_groups.isna().sum()

user_id    0
group      0
dtype: int64

In [13]:
df_purchases = pd.read_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-l-petrova/Final Project/Проект_4_purchases.csv', sep=',')

In [14]:
df_purchases.head()

,user_id,revenue
0,4412,1000.0
1,9000,701.0
2,9001,700.0
3,9002,702.0
4,5790,1002.0


In [15]:
df_purchases.shape[0]

541

In [16]:
df_purchases.isna().sum()

user_id    0
revenue    0
dtype: int64

In [17]:
df_purchases.dtypes

user_id      int64
revenue    float64
dtype: object

Для начала объединим все таблицы. 

In [18]:
df_merge = df_act_users.merge(df_groups, how="right", on=['user_id'])

In [19]:
df_merge.shape

(74576, 5)

In [20]:
df_merge_all = df_merge.merge(df_purchases, how="outer", on=['user_id'])

In [21]:
df_merge_all.head()

,user_id,country,platform,sex,group,revenue
0,9149,NaN,NaN,NaN,B,NaN
1,9150,NaN,NaN,NaN,B,NaN
2,9151,NaN,NaN,NaN,A,NaN
3,9152,NaN,NaN,NaN,B,NaN
4,9153,NaN,NaN,NaN,B,NaN


In [22]:
df_merge_all.shape

(74576, 6)

In [23]:
df_merge_all.revenue.sum()

578737.0

In [24]:
df_merge_all.dtypes

user_id       int64
country      object
platform     object
sex          object
group        object
revenue     float64
dtype: object

In [25]:
# посмотрим доход по группам A и В
df_merge_all.groupby('group') \
    .agg({'revenue': 'sum'})

,revenue
group,
A,93162.0
B,485575.0


In [28]:
# посмотрим среднее значение и медиану дохода по группам
df_merge_all.groupby('group').agg({'revenue': ["median", "mean"]})

revenue             
       median         mean
group                     
A       698.0   870.672897
B       944.0  1118.836406

In [29]:
# построим графики распределния дохода по группам
sns.distplot(df_control.revenue, kde=False, bins=50)
sns.distplot(df_test.revenue, kde=False, bins=50)


NameError: name 'df_control' is not defined

По графику видим, что нормального распределения данных не видим, то есть опираться на среднее значение данных не будем.
Далее будем смотреть данные по медиане, так как в таких данных более достоверной является показатель медианы.

In [30]:
# На +35% процентов увеличился средний (по медиане) доход в тестовой группе по сравнению с контрольной. 
round((944.0-698.0)*100/698.0, 2)

35.24

In [31]:
# посмотрим средний показатель дохода в разрезе по стране, платофрме и полу. Видим, что в тестовой группе доход увеличился.
revenue_by_country = (
    df_merge_all.groupby(["group", "country"], as_index=False)
    .agg({"revenue": "median"})
    .pivot(index="group", columns="country", values="revenue")
)
revenue_by_country.head()

country,Russia,Sweden
group,,
A,590.0,698.0
B,1001.0,1908.0


In [32]:
revenue_by_platform = (
    df_merge_all.groupby(["group", "platform"], as_index=False)
    .agg({"revenue": "median"})
    .pivot(index="group", columns="platform", values="revenue")
)
revenue_by_platform.head()

platform,android,ios
group,,
A,600.0,590.0
B,1151.0,1170.0


In [33]:
revenue_by_sex = (
    df_merge_all.groupby(["group", "sex"], as_index=False)
    .agg({"revenue": "median"})
    .pivot(index="group", columns="sex", values="revenue")
)
revenue_by_sex.head()

sex,female,male
group,,
A,590.0,600.0
B,1908.0,1000.5


In [34]:
# Отметим, что доход представлен не по всем пользователям. Поэтому для анализа отберём только пользователей, совершивших оплату:
df_merge_all_final = df_merge_all.query('revenue != "NaN"')
df_merge_all_final.head()

,user_id,country,platform,sex,group,revenue
60,9061,NaN,NaN,NaN,B,699.0
593,9036,NaN,NaN,NaN,B,300.0
594,9073,NaN,NaN,NaN,B,702.0
1722,9130,NaN,NaN,NaN,B,591.0
3910,9035,NaN,NaN,NaN,B,700.0


In [35]:
df_merge_all_final.shape

(541, 6)

Далее создаем отдельные дф по группам.

In [36]:
df_control = df_merge_all_final.query('group == "A"')

In [37]:
df_test = df_merge_all_final.query('group == "B"')

In [38]:
df_control.head()

,user_id,country,platform,sex,group,revenue
3919,9034,NaN,NaN,NaN,A,700.0
6556,9025,NaN,NaN,NaN,A,700.0
8008,9050,NaN,NaN,NaN,A,701.0
8014,9041,NaN,NaN,NaN,A,700.0
11234,1673,Sweden,android,male,A,2899.0


In [39]:
df_control.shape

(107, 6)

In [40]:
# удостоверяемся, что все значения пользователей уникальные
df_control.user_id.nunique()

107

In [41]:
df_control.revenue.sum()

93162.0

In [42]:
df_test.head()

,user_id,country,platform,sex,group,revenue
60,9061,NaN,NaN,NaN,B,699.0
593,9036,NaN,NaN,NaN,B,300.0
594,9073,NaN,NaN,NaN,B,702.0
1722,9130,NaN,NaN,NaN,B,591.0
3910,9035,NaN,NaN,NaN,B,700.0


In [43]:
df_test.shape

(434, 6)

In [44]:
df_test.user_id.nunique()

434

In [45]:
df_test.revenue.sum()

485575.0

In [46]:
round(df_test.user_id.nunique()/ df_control.user_id.nunique())

4

In [47]:
df_test.revenue.sum()/df_control.revenue.sum()

5.2121573173611555

#### По результатам видим, что в тестовой группе доход получился больше тестовой в 5 раз.
#### То есть в тестовой группе, где предлагалась новая модель оплаты коллекций шаблонов,доход больше, но надо иметь ввиду, что и пользователей в тестовой группе также больше в 4 раза, нежели в контрольной.
#### Возможно изменения в механике влияют в положительную сторону, соотвественно нулевая гипотеза должна отклоняться, по статистическим данным проверим, так ли это.

#### Теперь давайте изучим наши распределения. 
#### Выборка у нас в каждой группе небольшая - 107 и 434.
#### Проверим выполнение условий на нормальность распределения.
#### Сделаем анализ по критерию Шапиро-Уилко. 

In [48]:
ss.shapiro(df_control.revenue)

ShapiroResult(statistic=0.7201215028762817, pvalue=5.480257090142993e-13)

In [49]:
ss.shapiro(df_test.revenue)

ShapiroResult(statistic=0.8677289485931396, pvalue=9.681641387832235e-19)

In [50]:
# Сделаем семплирование перед применением критерия при проверке на нормальность (и для теста, и для контроля), 
# так как на больших выборках можем получить неверные результаты.
ss.shapiro(df_control.revenue.sample(100, random_state=17))

ShapiroResult(statistic=0.7246447801589966, pvalue=2.105227233858975e-12)

In [51]:
ss.shapiro(df_test.revenue.sample(100, random_state=17))

ShapiroResult(statistic=0.8417040109634399, pvalue=5.925657298178066e-09)

#### Нулевая гипотеза в Тесте Шапиро-Уилко (если pval>0.5) говорит нам о нормальном распределении. В нашем случае (с семплированием и без него) получились pval<0, что говорит нам о ненормальном распределении в обеих группах, что также было подтверждено графиком.

Далее проверим дисперсию внутри наших группс помощью теста Левена. Для применения стат метода ttest они должны быть примерно одинаковы (требование гомогенности дисперсий). 

In [52]:
ss.levene(df_control.revenue,df_test.revenue)

LeveneResult(statistic=4.465304117356759, pvalue=0.03504894627055691)

#### pvalue=0.035, что меньше заданного уровня значимости 0,05, значит гипотеза о равенстве дисперсий неверна. Поэтому далее для стат анализа мы не можем применить критерий ttest. Можем прологарифмировать данные выборки для приведения их в вид нормального распределения и далее провести т-тест с интерпретацией результатов.
#### А можем без этого этапа пойти далее и использовать для стат анализа Бутстрап
При применении бустрапа уровень значимости возьмем 95%. Выявим p_value среднего и медианы в двух случаях.

In [53]:
A = df_control.revenue
B = df_test.revenue

In [54]:
# Объявим функцию, которая позволит проверять гипотезы с помощью бутстрапа
def get_bootstrap(
    data_column_1 = A,  # числовые значения первой выборки
    data_column_2 = B,  # числовые значения второй выборки
    boot_it=1000,  # количество бутстрэп-подвыборок
    statistic=np.mean,  # интересующая нас статистика
    bootstrap_conf_level=0.95  # уровень значимости, на базе которого проверяем гипотезу
):
    boot_len = max([len(data_column_1), len(data_column_2)])
    boot_data = []
    for i in range(boot_it):  # извлекаем подвыборки
        samples_1 = data_column_1.sample(
            boot_len, 
            replace=True  # параметр возвращения
        ).values
        samples_2 = data_column_2.sample(
            boot_len, 
            replace = True
        ).values
        
        boot_data.append(statistic(samples_1 - samples_2))  # mean() - применяем статистику
    
    pd_boot_data = pd.DataFrame(boot_data)        
    left_quant = (1 - bootstrap_conf_level)/ 2
    right_quant = 1 - (1 - bootstrap_conf_level) / 2
    ci = pd_boot_data.quantile([left_quant, right_quant])

    p_1 = norm.cdf(
        x=0, 
        loc=np.mean(boot_data), 
        scale=np.std(boot_data)
    )
    p_2 = norm.cdf(
        x = 0, 
        loc = -np.mean(boot_data), 
        scale = np.std(boot_data)
    )
    p_value = min(p_1, p_2) * 2

    # Визуализация
#     plt.hist(pd_boot_data[0], bins = 50)
    
#     plt.style.use('ggplot')
#     plt.vlines(ci, ymin=0 ymax=50, linestyle='--')
#     plt.xlabel('boot_data')
#     plt.ylabel('frequency')
#     plt.title("Histogram of boot_data")
#     plt.show()
       
    return {"boot_data": boot_data, 
            "ci": ci, 
            "p_value": p_value}
    

In [55]:
# здесь не поняла, в чем ошибка, не выводит результат.
print(f'p_value:{p_value}, ci:{ci}')

NameError: name 'p_value' is not defined

In [56]:
# Проверим гипотезу на непараметрическом тесте Манна-Уитни:
res_mw = mannwhitneyu(A, B, alternative='two-sided')
res_mw

MannwhitneyuResult(statistic=17699.5, pvalue=0.0001364710277394854)

In [57]:
# p-значение < 0.05 говорит о том,что статистически значимые различия в группах присутствуют.

In [58]:
for i in range(10):
    res_b_npmean = get_bootstrap(A, B)
    res_b_npmedian = get_bootstrap(A, B, statistic=np.median)
    
    print(f'Boot mean: {res_b_npmean["p_value"]}, Boot median: {res_b_npmedian["p_value"]}, MW {res_mw.pvalue}')

Boot mean: 2.37116330681496e-06, Boot median: 0.0004194219301573444, MW 0.0001364710277394854
Boot mean: 1.5922671539970113e-06, Boot median: 0.0008645283067397059, MW 0.0001364710277394854
Boot mean: 3.153222246232769e-06, Boot median: 0.0005412613383765052, MW 0.0001364710277394854
Boot mean: 1.9377005113325514e-06, Boot median: 0.0007247949398961117, MW 0.0001364710277394854
Boot mean: 5.04230320052471e-06, Boot median: 0.000386035198818471, MW 0.0001364710277394854
Boot mean: 2.2728360385108224e-06, Boot median: 0.0009219799947354014, MW 0.0001364710277394854
Boot mean: 8.177108907396979e-07, Boot median: 0.000542874090637301, MW 0.0001364710277394854
Boot mean: 3.57170602939344e-06, Boot median: 0.0008857402548965842, MW 0.0001364710277394854
Boot mean: 8.221992493298402e-06, Boot median: 0.0006223819441865627, MW 0.0001364710277394854
Boot mean: 3.7939423225684097e-06, Boot median: 0.0011266222639283417, MW 0.0001364710277394854


#### Результат можем интерпретировать как:
#### у нас есть средний/медианный доход по платящим пользователям по 2 группам  А и В, который в результате исследования данных с помощью Бустрап показал, что статистически значимые различия метрики ARPPU присутствуют. 
#### Как обосновали: делаем бустрап-распределение параметра для групп, получаем распределение разниц значений по доходу, в получившимся распределении разниц  видим, что в бустрапированных выборках 0 (ноль) не входит в доверительный интервал, p_value < 0.05, следовательно, сравнивая средние значения и медиану, мы можем говорить, что группы отличаются и нулевая гипотеза о равенстве значений параметра между 2мя группами эксперимента отклоняется, соответственно, выкатываем новую модель.


#### Далее анализируем изменение статист. значимость  конверсии, есть ли рост конверсии от изменения модели оплаты.
#### В случае, когда мы измеряем только конверсию, то  имеем дело с бинарной случайной величиной, которая принимает только два значения: посетитель “сконвертировался” и “не сконвертировался”.


In [59]:
# в этой таблице есть все пользователи, заходившие в день эксперимента
df_merge_all.head()

,user_id,country,platform,sex,group,revenue
0,9149,NaN,NaN,NaN,B,NaN
1,9150,NaN,NaN,NaN,B,NaN
2,9151,NaN,NaN,NaN,A,NaN
3,9152,NaN,NaN,NaN,B,NaN
4,9153,NaN,NaN,NaN,B,NaN


In [60]:
df_merge_all["convertation"] = df_merge_all.revenue.apply(lambda x: 0 if np.isnan(x) else 1)
df_merge_all.head()

,user_id,country,platform,sex,group,revenue,convertation
0,9149,NaN,NaN,NaN,B,NaN,0
1,9150,NaN,NaN,NaN,B,NaN,0
2,9151,NaN,NaN,NaN,A,NaN,0
3,9152,NaN,NaN,NaN,B,NaN,0
4,9153,NaN,NaN,NaN,B,NaN,0


In [61]:
# в этой таблице есть только пользователи, совершившие покупку
df_merge_all_final.head()

,user_id,country,platform,sex,group,revenue
60,9061,NaN,NaN,NaN,B,699.0
593,9036,NaN,NaN,NaN,B,300.0
594,9073,NaN,NaN,NaN,B,702.0
1722,9130,NaN,NaN,NaN,B,591.0
3910,9035,NaN,NaN,NaN,B,700.0


In [62]:
# кол-во заплативших юзеров в группе А
df_merge_all_final.query('group == "A"').shape[0]

107

In [63]:
# кол-во всех юзеров в группе А
df_merge_all.query('group == "A"').shape[0]

14693

In [64]:
# кол-во заплативших юзеров в группе В
df_merge_all_final.query('group == "B"').shape[0]

434

In [65]:
# кол-во всех юзеров в группе В
df_merge_all.query('group == "B"').shape[0]

59883

In [66]:
# найдем конверсию по группам А и В
CR_A = 100*df_merge_all_final.query('group == "A"').shape[0]/ df_merge_all.query('group == "A"').shape[0]
CR_A

0.7282379364323147

In [67]:
CR_B = 100*df_merge_all_final.query('group == "B"').shape[0]/ df_merge_all.query('group == "B"').shape[0]
CR_B

0.7247465891822387

In [68]:
# видим, что они отличаются друг от друга незначительно. Оценим статистическую значимость изменения конверсии между тестовой 
# и контрольной группами с помощью непараметрического метода хи-квадрат Пирсона.

In [69]:
# Создаем сводную таблицу или таблицу сопряженности
pd.crosstab(df_merge_all.convertation, df_merge_all.group)

group,A,B
convertation,,
0,14586,59449
1,107,434


In [70]:
chi2, prob, df, expected = ss.chi2_contingency(pd.crosstab(df_merge_all.convertation, df_merge_all.group))
output = "test Statistics: {}\ndegrees of freedom: {}\np-value: {}\n"
print(output.format(chi2, df, prob))
print(expected)

test Statistics: 0.0
degrees of freedom: 1
p-value: 1.0

[[14586.41191536 59448.58808464]
 [  106.58808464   434.41191536]]


In [71]:
# как видим, статистически значимых различий в изменении конверсии нет.

In [72]:
# можно провести более глубокий анализ и посомтреть изменение конверсии в группах в разрезе по  стране, платформе и полу.

In [73]:
CR_by_country = (
    df_merge_all.groupby(["group", "country"], as_index=False) \
    .agg({"convertation": "sum"}) \
    .pivot(index="group", columns="country", values="convertation")
)
CR_by_country.head()

country,Russia,Sweden
group,,
A,43,35
B,197,117


In [74]:
chi2, prob, df, expected = ss.chi2_contingency(CR_by_country)
output = "test Statistics: {}\ndegrees of freedom: {}\np-value: {}\n"
print(output.format(chi2, df, prob))
print(expected)

test Statistics: 1.2206722710745501
degrees of freedom: 1
p-value: 0.26922875889692416

[[ 47.75510204  30.24489796]
 [192.24489796 121.75510204]]


In [75]:
CR_by_platform = (
    df_merge_all.groupby(["group", "platform"], as_index=False)
    .agg({"convertation": "sum"})
    .pivot(index="group", columns="platform", values="convertation")
)
CR_by_platform.head()

platform,android,ios
group,,
A,39,39
B,169,145


In [76]:
chi2, prob, df, expected = ss.chi2_contingency(CR_by_platform)
output = "test Statistics: {}\ndegrees of freedom: {}\np-value: {}\n"


print(output.format( chi2, df, prob))
print(expected)

test Statistics: 0.2290045428798804
degrees of freedom: 1
p-value: 0.6322629292404002

[[ 41.3877551  36.6122449]
 [166.6122449 147.3877551]]


In [77]:
CR_by_sex = (
    df_merge_all.groupby(["group", "sex"], as_index=False)
    .agg({"convertation": "sum"})
    .pivot(index="group", columns="sex", values="convertation")
)
CR_by_sex.head()

sex,female,male
group,,
A,37,41
B,154,160


In [78]:
chi2, prob, df, expected = ss.chi2_contingency(CR_by_sex)
output = "test Statistics: {}\ndegrees of freedom: {}\np-value: {}\n"


print(output.format( chi2, df, prob))
print(expected)

test Statistics: 0.016344166283766903
degrees of freedom: 1
p-value: 0.8982721875127574

[[ 38.00510204  39.99489796]
 [152.99489796 161.00510204]]


In [79]:
# анализ показал, что стат.значимости нет в изменении конверсии в разрезе страны, платформы и пола.

### Вывод: в качестве ключевых метрик были выбраны чек (доход) с одного пользователя и конверсия. Результаты методов стат обработки показывают, что:
### 1. значимость различий в  чеках пользователей между группами А и В существенна. То есть есть смысл далее использовать новую модель оплаты. Но
### 2. значимость различия в изменении конверсии между группами А и В отсутствует, что говорит о нецелесообразности выкатывания новой модели оплаты.
### Отсюда можно предположить, что нельзя однозначно сказать, будем далее использовать новую модель или нет. Нужно поработать вначале с данными, приравнять число людей в группах А и В например.

# Задание 2

Одной из основных задач аналитика является не только построение моделей, но и создание дашбордов, которые позволяют отслеживать изменения метрик и принимать на их основе оптимальные решения. Ваш руководитель хочет узнать, как обстоят дела с использованием приложения и вовлечённостью пользователей, и очень просит спроектировать ему дашборд.

active_users – информация о пользователях, которые посещали приложение во время эксперимента. 

groups – разбивка пользователей на контрольную (А) и тестовую (В) группы. 

purchases – данные о транзакциях (оплатах) пользователей приложения во время эксперимента 

In [91]:
df_act_users['active_users']=1
df_act_users.head()

,user_id,country,platform,sex,active_users
0,8029,Russia,ios,male,1
1,1735,Sweden,ios,male,1
2,7697,Russia,android,male,1
3,1421,Sweden,android,female,1
4,1704,Russia,ios,male,1


In [92]:
df_act_users

,user_id,country,platform,sex,active_users
0,8029,Russia,ios,male,1
1,1735,Sweden,ios,male,1
2,7697,Russia,android,male,1
3,1421,Sweden,android,female,1
4,1704,Russia,ios,male,1
...,...,...,...,...,...
8336,2417,Sweden,ios,female,1
8337,3755,Russia,ios,male,1
8338,4594,Sweden,android,female,1
8339,7430,Sweden,ios,female,1


In [93]:
df_act_users.user_id.nunique()

8341

In [94]:
df_purchases['converted_users']=1
df_purchases.head()

,user_id,revenue,converted_users
0,4412,1000.0,1
1,9000,701.0,1
2,9001,700.0,1
3,9002,702.0,1
4,5790,1002.0,1


In [95]:
df_purchases.shape[0]

541

In [96]:
df_merge_2 = df_act_users.merge(df_groups, how="right", on=['user_id']).merge(df_purchases, how="outer", on=['user_id'])
df_merge_2.head()

,user_id,country,platform,sex,active_users,group,revenue,converted_users
0,9149,NaN,NaN,NaN,NaN,B,NaN,NaN
1,9150,NaN,NaN,NaN,NaN,B,NaN,NaN
2,9151,NaN,NaN,NaN,NaN,A,NaN,NaN
3,9152,NaN,NaN,NaN,NaN,B,NaN,NaN
4,9153,NaN,NaN,NaN,NaN,B,NaN,NaN


In [97]:
df_merge_2.shape[0]

74576

In [98]:
df_merge_2["active_users"].fillna(0, inplace=True)
df_merge_2["converted_users"].fillna(0, inplace=True)
df_merge_2 = df_merge_2.astype({"active_users": "int", "converted_users": "int"})
df_merge_2.head()

,user_id,country,platform,sex,active_users,group,revenue,converted_users
0,9149,NaN,NaN,NaN,0,B,NaN,0
1,9150,NaN,NaN,NaN,0,B,NaN,0
2,9151,NaN,NaN,NaN,0,A,NaN,0
3,9152,NaN,NaN,NaN,0,B,NaN,0
4,9153,NaN,NaN,NaN,0,B,NaN,0


In [106]:
df_merge_2.active_users.sum()

8341

In [110]:
df_merge_2.revenue.sum()

578737.0

In [109]:
df_merge_2.converted_users.sum()

541

In [102]:
df_merge_2.to_csv('data_for_dashboard.csv', index=False)

#### Дашборд можно посмотреть по ссылке:
https://public.tableau.com/views/final_project_2_16539007514400/Dashboard2?:language=en-US&publish=yes&:display_count=n&:origin=viz_share_link